<a href="https://colab.research.google.com/github/havhje/naturregnskap_jupityr/blob/main/Dashboard_nat_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Importer biblioteker og stil-oppsett
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interactive, FloatSlider, Dropdown, fixed, HBox, VBox, Output, Label, Tab, RadioButtons, Button, Text, Checkbox
from IPython.display import Markdown, display, HTML, clear_output
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mtick

# Style-innstillinger
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# Fargekart for naturtilstand
naturkvalitet_cmap = LinearSegmentedColormap.from_list('naturkvalitet',
                                                      ['#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15'],
                                                      N=256)

# Fargekart for kompensasjon
komp_cmap = LinearSegmentedColormap.from_list('kompensasjon',
                                            ['#f2f0f7', '#cbc9e2', '#9e9ac8', '#6a51a3'],
                                            N=256)

# Fargekart for tap
tap_cmap = LinearSegmentedColormap.from_list('tap',
                                           ['#eff3ff', '#bdd7e7', '#6baed6', '#2171b5'],
                                           N=256)

# Fargekart for netto endring
change_cmap = LinearSegmentedColormap.from_list('change',
                                               ['#d73027', '#fc8d59', '#fee090', '#e0f3f8', '#91bfdb', '#4575b4'],
                                               N=256)

In [2]:
# Cell 2: Definer referanseverdier og konstanter
# Definer verdier for naturkvalitet basert på tabellene
naturkvalitet_øko = [
    ('Verneområder/utvalgte naturtyper', 5.0),
    ('Økologiske funksjonsområder for prioriterte arter', 5.0),
    ('Naturtyper - Svært høy lokalitetskvalitet', 4.0),
    ('Naturtyper - Høy lokalitetskvalitet', 3.0),
    ('Naturtyper - Moderat lokalitetskvalitet', 2.2),
    ('Naturtyper - Lav lokalitetskvalitet', 2.0),
    ('Naturtyper - Svært lav lokalitetskvalitet', 1.0),
    ('Øvrig natur - Høy kvalitet', 2.0),
    ('Øvrig natur - Moderat kvalitet', 1.0),
    ('Øvrig natur - Lav kvalitet', 0.5),
    ('Sterkt endrede arealer', 0.0)
]

naturkvalitet_funk = [
    ('Villrein - Nasjonalt villreinområde', 4.0),
    ('Villrein - Fastsatte randområder', 3.0),
    ('Villrein - Fastsatte bygdenære områder', 2.2),
    ('Funksjonsområder - Prioritert art, fugl og pattedyr', 5.0),
    ('Funksjonsområder - Svært høy kvalitet', 4.0),
    ('Funksjonsområder - Høy kvalitet', 3.0),
    ('Funksjonsområder - Moderat kvalitet', 2.2),
    ('Funksjonsområder - Lav kvalitet', 2.0),
    ('Funksjonsområder - Svært lav kvalitet', 1.0)
]

naturkvalitet_vann = [
    ('Svært høy kvalitet', 4.0),
    ('Høy kvalitet', 3.0),
    ('Moderat kvalitet', 2.2),
    ('Lav kvalitet', 2.0),
    ('Svært lav kvalitet', 1.0)
]

# Definer verdier for forvaltningsinteresse
forvaltningsinteresse_options = [
    ('Verneområder', 5.0),
    ('Utvalgte naturtyper', 5.0),
    ('Økologiske funksjonsområder for prioriterte arter', 5.0),
    ('CR og EN naturtype', 4.0),
    ('VU naturtype', 3.0),
    ('NT naturtype/Sentral økosystem funksjon/Dårlig kartlagt', 2.0),
    ('Øvrig natur og Sterkt endret areal', 1.0),
    ('Prioritert art (fugl og pattedyr)', 5.0),
    ('CR og EN art', 4.0),
    ('VU art, andre spesielt hensynskrevende arter', 3.0),
    ('NT art og ansvarsarter', 2.0),
    ('Fredede og prioriterte arter, EN og CR arter', 4.0)
]

# Definer verdier for påvirkning
påvirkning_options = [
    ('Ingen påvirkning', 0.0),
    ('Ubetydelig påvirkning', 0.1),
    ('Noe forringet', 0.2),
    ('Forringet', 0.4),
    ('Sterkt forringet', 0.6),
    ('Svært sterkt forringet', 0.8)
]

# Definer verdier for risikofaktorer
risiko_avstand_options = [
    ('Samme sted (0 km)', 1.0),
    ('Nærområdet (0-3 km)', 0.9),
    ('Samme kommune (3-10 km)', 0.8),
    ('Samme region (10-30 km)', 0.7),
    ('Utenfor regionen (>30 km)', 0.6)
]

risiko_tid_options = [
    ('Ingen tidsforsinkelse', 1.0),
    ('Kort tidsforsinkelse (0-5 år)', 0.9),
    ('Middels tidsforsinkelse (5-15 år)', 0.8),
    ('Lang tidsforsinkelse (15-50 år)', 0.7),
    ('Svært lang tidsforsinkelse (>50 år)', 0.6)
]

risiko_vanskelighet_options = [
    ('Svært enkel', 1.0),
    ('Enkel', 0.9),
    ('Middels', 0.8),
    ('Vanskelig', 0.7),
    ('Svært vanskelig', 0.6)
]

In [3]:
# Cell 3: Beregningsfunksjoner for naturpoeng
# --- Del 1: Beregningsfunksjoner ---
def beregn_naturpoeng_før(utstrekning, naturkvalitet, forvaltningsinteresse):
    """Beregner naturpoeng før inngrep"""
    return utstrekning * naturkvalitet * forvaltningsinteresse

def beregn_naturpoeng_tapt(tapte_utstrekning, rest_utstrekning, naturkvalitet,
                          forvaltningsinteresse, påvirkning):
    """Beregner naturpoeng tapt, både direkte og indirekte"""
    direkte_tap = tapte_utstrekning * naturkvalitet * forvaltningsinteresse
    indirekte_tap = (rest_utstrekning * naturkvalitet * forvaltningsinteresse) * påvirkning
    return direkte_tap + indirekte_tap, direkte_tap, indirekte_tap

def beregn_naturpoeng_skapt(komp_utstrekning, komp_naturkvalitet, komp_forvaltningsinteresse,
                           risiko_avstand, risiko_tid, risiko_vanskelighet,
                           er_offsite=False, original_areal=None):
    """Beregner naturpoeng skapt gjennom kompensasjonstiltak"""
    skapte_poeng = (komp_utstrekning * komp_naturkvalitet * komp_forvaltningsinteresse *
                    risiko_avstand * risiko_tid * risiko_vanskelighet)

    if er_offsite and original_areal is not None:
        return max(0, skapte_poeng - original_areal)

    return skapte_poeng

In [5]:
# Cell 4: Funksjon for å importere eksempeldata
def importer_eksempeldata():
    """Importerer eksempeldata for vindprosjektet med 50 delområder"""
    # Opprett datastruktur for økosystemareal (25 delområder)
    oko_data = [
        {"ID": "Ø1", "Navn": "Øko_1_Kystlynghei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Høy lokalitetskvalitet",
         "Utstrekning": 87, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 19, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 38, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 2.2,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø2", "Navn": "Øko_2_Boreal_hei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 132, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 42, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 75, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.8,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø3", "Navn": "Øko_3_Semi-naturlig_eng", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 68, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 12, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 20, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø4", "Navn": "Øko_4_Oseanisk_torvmyr", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Høy lokalitetskvalitet",
         "Utstrekning": 95, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 32, "Påvirkning": 0.4,
         "Komp_type": "Off-site", "Komp_utstrekning": 96, "Komp_naturkvalitet": 2.5, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.8, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.7},

        {"ID": "Ø5", "Navn": "Øko_5_Kystnedbørsmyr", "Delregnskap": "Økosystemareal", "Naturtype": "Verneområder/utvalgte naturtyper",
         "Utstrekning": 42, "Naturkvalitet": 5.0, "Forvaltningsinteresse": 5.0, "Tapt_direkte": 0, "Påvirkning": 0.1,
         "Komp_type": "-", "Komp_utstrekning": 0, "Komp_naturkvalitet": 0, "Komp_forvaltningsinteresse": 0,
         "Risiko_avstand": 0, "Risiko_tid": 0, "Risiko_vanskelighet": 0},

        {"ID": "Ø6", "Navn": "Øko_6_Gammel_granskog", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Høy lokalitetskvalitet",
         "Utstrekning": 110, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 38, "Påvirkning": 0.6,
         "Komp_type": "On-site", "Komp_utstrekning": 95, "Komp_naturkvalitet": 2.5, "Komp_forvaltningsinteresse": 3.2,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.6},

        {"ID": "Ø7", "Navn": "Øko_7_Kalkrik_fjellhei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Svært høy lokalitetskvalitet",
         "Utstrekning": 56, "Naturkvalitet": 4.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 24, "Påvirkning": 0.6,
         "Komp_type": "On-site", "Komp_utstrekning": 48, "Komp_naturkvalitet": 3.2, "Komp_forvaltningsinteresse": 3.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "Ø8", "Navn": "Øko_8_Rik_blandingsskog", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 142, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 52, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 110, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 1.0, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø9", "Navn": "Øko_9_Boreonemoral_regnskog", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Høy lokalitetskvalitet",
         "Utstrekning": 72, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 0, "Påvirkning": 0.1,
         "Komp_type": "-", "Komp_utstrekning": 0, "Komp_naturkvalitet": 0, "Komp_forvaltningsinteresse": 0,
         "Risiko_avstand": 0, "Risiko_tid": 0, "Risiko_vanskelighet": 0},

        {"ID": "Ø10", "Navn": "Øko_10_Kystlynghei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Lav lokalitetskvalitet",
         "Utstrekning": 128, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 42, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 65, "Komp_naturkvalitet": 1.6, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        # Flere økosystemareal (11-25)
        {"ID": "Ø11", "Navn": "Øko_11_Boreal_hei", "Delregnskap": "Økosystemareal", "Naturtype": "Øvrig natur - Høy kvalitet",
         "Utstrekning": 85, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 1.0, "Tapt_direkte": 18, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 28, "Komp_naturkvalitet": 1.5, "Komp_forvaltningsinteresse": 1.0,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.9, "Risiko_vanskelighet": 0.9},

        {"ID": "Ø12", "Navn": "Øko_12_Semi-naturlig_eng", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 62, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 14, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 28, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø13", "Navn": "Øko_13_Kystnedbørsmyr", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Lav lokalitetskvalitet",
         "Utstrekning": 176, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 48, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 85, "Komp_naturkvalitet": 1.6, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "Ø14", "Navn": "Øko_14_Gammel_granskog", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 92, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 28, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 45, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø15", "Navn": "Øko_15_Rik_blandingsskog", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Høy lokalitetskvalitet",
         "Utstrekning": 65, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 22, "Påvirkning": 0.4,
         "Komp_type": "Off-site", "Komp_utstrekning": 42, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.7, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "Ø16", "Navn": "Øko_16_Oseanisk_torvmyr", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Lav lokalitetskvalitet",
         "Utstrekning": 158, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 1.0, "Tapt_direkte": 38, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 58, "Komp_naturkvalitet": 1.6, "Komp_forvaltningsinteresse": 1.0,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.9, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø17", "Navn": "Øko_17_Kystlynghei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 118, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 35, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 65, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø18", "Navn": "Øko_18_Kalkrik_fjellhei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Svært høy lokalitetskvalitet",
         "Utstrekning": 38, "Naturkvalitet": 4.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 0, "Påvirkning": 0.1,
         "Komp_type": "-", "Komp_utstrekning": 0, "Komp_naturkvalitet": 0, "Komp_forvaltningsinteresse": 0,
         "Risiko_avstand": 0, "Risiko_tid": 0, "Risiko_vanskelighet": 0},

        {"ID": "Ø19", "Navn": "Øko_19_Rik_fjellgrashei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Høy lokalitetskvalitet",
         "Utstrekning": 78, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 18, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 32, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "Ø20", "Navn": "Øko_20_Boreal_hei", "Delregnskap": "Økosystemareal", "Naturtype": "Øvrig natur - Moderat kvalitet",
         "Utstrekning": 185, "Naturkvalitet": 1.0, "Forvaltningsinteresse": 1.0, "Tapt_direkte": 62, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 98, "Komp_naturkvalitet": 1.0, "Komp_forvaltningsinteresse": 1.0,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.9, "Risiko_vanskelighet": 0.9},

        {"ID": "Ø21", "Navn": "Øko_21_Kystlynghei", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 96, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 32, "Påvirkning": 0.4,
         "Komp_type": "Off-site", "Komp_utstrekning": 75, "Komp_naturkvalitet": 2.0, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.7, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "Ø22", "Navn": "Øko_22_Semi-naturlig_eng", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Lav lokalitetskvalitet",
         "Utstrekning": 78, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 22, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 38, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø23", "Navn": "Øko_23_Boreonemoral_regnskog", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Svært høy lokalitetskvalitet",
         "Utstrekning": 42, "Naturkvalitet": 4.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 0, "Påvirkning": 0.0,
         "Komp_type": "-", "Komp_utstrekning": 0, "Komp_naturkvalitet": 0, "Komp_forvaltningsinteresse": 0,
         "Risiko_avstand": 0, "Risiko_tid": 0, "Risiko_vanskelighet": 0},

        {"ID": "Ø24", "Navn": "Øko_24_Kystnedbørsmyr", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Høy lokalitetskvalitet",
         "Utstrekning": 68, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 18, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 32, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "Ø25", "Navn": "Øko_25_Gammel_granskog", "Delregnskap": "Økosystemareal", "Naturtype": "Naturtyper - Moderat lokalitetskvalitet",
         "Utstrekning": 112, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 42, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 76, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7}
    ]

    funk_data = [
        {"ID": "F1", "Navn": "Funk_1_Kongeørn", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Høy kvalitet",
         "Utstrekning": 285, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 86, "Påvirkning": 0.8,
         "Komp_type": "On-site", "Komp_utstrekning": 172, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 3.2,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.6},

        {"ID": "F2", "Navn": "Funk_2_Havørn", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Prioritert art, fugl og pattedyr",
         "Utstrekning": 220, "Naturkvalitet": 5.0, "Forvaltningsinteresse": 5.0, "Tapt_direkte": 55, "Påvirkning": 0.8,
         "Komp_type": "Off-site", "Komp_utstrekning": 138, "Komp_naturkvalitet": 4.0, "Komp_forvaltningsinteresse": 4.5,
         "Risiko_avstand": 0.7, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.6},

        {"ID": "F3", "Navn": "Funk_3_Hubro", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Prioritert art, fugl og pattedyr",
         "Utstrekning": 180, "Naturkvalitet": 5.0, "Forvaltningsinteresse": 5.0, "Tapt_direkte": 45, "Påvirkning": 0.8,
         "Komp_type": "Off-site", "Komp_utstrekning": 120, "Komp_naturkvalitet": 4.0, "Komp_forvaltningsinteresse": 4.0,
         "Risiko_avstand": 0.7, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.6},

        {"ID": "F4", "Navn": "Funk_4_Storfugl", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Høy kvalitet",
         "Utstrekning": 248, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 68, "Påvirkning": 0.6,
         "Komp_type": "On-site", "Komp_utstrekning": 124, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "F5", "Navn": "Funk_5_Orrfugl", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Moderat kvalitet",
         "Utstrekning": 182, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 52, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 95, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "F6", "Navn": "Funk_6_Hjort", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Moderat kvalitet",
         "Utstrekning": 235, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 65, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 120, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "F7", "Navn": "Funk_7_Lirype", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Moderat kvalitet",
         "Utstrekning": 162, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 42, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 75, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "F8", "Navn": "Funk_8_Fjellrype", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Moderat kvalitet",
         "Utstrekning": 198, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 56, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 105, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "F9", "Navn": "Funk_9_Jaktfalk", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Høy kvalitet",
         "Utstrekning": 145, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 38, "Påvirkning": 0.6,
         "Komp_type": "Off-site", "Komp_utstrekning": 95, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 3.2,
         "Risiko_avstand": 0.7, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.7},

        {"ID": "F10", "Navn": "Funk_10_Elg", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Lav kvalitet",
         "Utstrekning": 265, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 78, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 145, "Komp_naturkvalitet": 1.6, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "F11", "Navn": "Funk_11_Vandrefalk", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Høy kvalitet",
         "Utstrekning": 165, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 42, "Påvirkning": 0.6,
         "Komp_type": "Off-site", "Komp_utstrekning": 84, "Komp_naturkvalitet": 2.4, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.7, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.7},

        {"ID": "F12", "Navn": "Funk_12_Gaupe", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Lav kvalitet",
         "Utstrekning": 295, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 85, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 165, "Komp_naturkvalitet": 1.6, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "F13", "Navn": "Funk_13_Rødrev", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Lav kvalitet",
         "Utstrekning": 275, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 1.0, "Tapt_direkte": 72, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 125, "Komp_naturkvalitet": 1.6, "Komp_forvaltningsinteresse": 1.0,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.9, "Risiko_vanskelighet": 0.9},

        {"ID": "F14", "Navn": "Funk_14_Mår", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Moderat kvalitet",
         "Utstrekning": 168, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 48, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 85, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "F15", "Navn": "Funk_15_Kongeørn", "Delregnskap": "Økologiske funksjonsområder", "Naturtype": "Funksjonsområder - Svært høy kvalitet",
         "Utstrekning": 142, "Naturkvalitet": 4.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 42, "Påvirkning": 0.8,
         "Komp_type": "Off-site", "Komp_utstrekning": 105, "Komp_naturkvalitet": 3.2, "Komp_forvaltningsinteresse": 3.2,
         "Risiko_avstand": 0.7, "Risiko_tid": 0.7, "Risiko_vanskelighet": 0.6}
    ]
    # Cell 6: Eksempeldata - vannforekomster
    # 3. Vannforekomster (10 delområder)
    vann_data = [
        {"ID": "V1", "Navn": "Vann_1_Fjellbekk", "Delregnskap": "Vannforekomster", "Naturtype": "Høy kvalitet",
         "Utstrekning": 345, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 28, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 42, "Komp_naturkvalitet": 2.5, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "V2", "Navn": "Vann_2_Myrbekk", "Delregnskap": "Vannforekomster", "Naturtype": "Moderat kvalitet",
         "Utstrekning": 425, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 36, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 68, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "V3", "Navn": "Vann_3_Kystvassdrag", "Delregnskap": "Vannforekomster", "Naturtype": "Svært høy kvalitet",
         "Utstrekning": 265, "Naturkvalitet": 4.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 22, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 42, "Komp_naturkvalitet": 3.2, "Komp_forvaltningsinteresse": 3.2,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "V4", "Navn": "Vann_4_Innsjø", "Delregnskap": "Vannforekomster", "Naturtype": "Høy kvalitet",
         "Utstrekning": 680, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 0, "Påvirkning": 0.1,
         "Komp_type": "-", "Komp_utstrekning": 0, "Komp_naturkvalitet": 0, "Komp_forvaltningsinteresse": 0,
         "Risiko_avstand": 0, "Risiko_tid": 0, "Risiko_vanskelighet": 0},

        {"ID": "V5", "Navn": "Vann_5_Tjern", "Delregnskap": "Vannforekomster", "Naturtype": "Moderat kvalitet",
         "Utstrekning": 185, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 0, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 0, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "V6", "Navn": "Vann_6_Kildebekk", "Delregnskap": "Vannforekomster", "Naturtype": "Svært høy kvalitet",
         "Utstrekning": 142, "Naturkvalitet": 4.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 12, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 24, "Komp_naturkvalitet": 3.2, "Komp_forvaltningsinteresse": 2.5,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "V7", "Navn": "Vann_7_Elv", "Delregnskap": "Vannforekomster", "Naturtype": "Høy kvalitet",
         "Utstrekning": 520, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 4.0, "Tapt_direkte": 45, "Påvirkning": 0.4,
         "Komp_type": "On-site", "Komp_utstrekning": 85, "Komp_naturkvalitet": 2.5, "Komp_forvaltningsinteresse": 3.2,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7},

        {"ID": "V8", "Navn": "Vann_8_Sig", "Delregnskap": "Vannforekomster", "Naturtype": "Lav kvalitet",
         "Utstrekning": 365, "Naturkvalitet": 2.0, "Forvaltningsinteresse": 1.0, "Tapt_direkte": 28, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 42, "Komp_naturkvalitet": 1.6, "Komp_forvaltningsinteresse": 1.0,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.9, "Risiko_vanskelighet": 0.9},

        {"ID": "V9", "Navn": "Vann_9_Myrbekk", "Delregnskap": "Vannforekomster", "Naturtype": "Moderat kvalitet",
         "Utstrekning": 285, "Naturkvalitet": 2.2, "Forvaltningsinteresse": 2.0, "Tapt_direkte": 24, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 38, "Komp_naturkvalitet": 1.8, "Komp_forvaltningsinteresse": 1.6,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.8},

        {"ID": "V10", "Navn": "Vann_10_Fjellbekk", "Delregnskap": "Vannforekomster", "Naturtype": "Høy kvalitet",
         "Utstrekning": 310, "Naturkvalitet": 3.0, "Forvaltningsinteresse": 3.0, "Tapt_direkte": 25, "Påvirkning": 0.2,
         "Komp_type": "On-site", "Komp_utstrekning": 45, "Komp_naturkvalitet": 2.5, "Komp_forvaltningsinteresse": 2.4,
         "Risiko_avstand": 0.9, "Risiko_tid": 0.8, "Risiko_vanskelighet": 0.7}
    ]

    # Slå sammen alle data til én samlet dataframe
    all_data = oko_data + funk_data + vann_data
    df = pd.DataFrame(all_data)

    # Fikser manglende data (null-verdier når kompensasjon ikke er relevant)
    df.fillna(0, inplace=True)

    # Kalkuler faktisk bruk av original_areal for off-site kompensasjon
    df['Original_areal'] = 0.0  # Standardverdi

    # Legg til beregnede verdier
    for i, row in df.iterrows():
        # For områder som faktisk har off-site kompensasjon, beregn en plausibel verdi
        if row['Komp_type'] == 'Off-site':
            df.at[i, 'Original_areal'] = row['Komp_utstrekning'] * 0.5  # Antatt verdi for illustrasjon

    return df

In [6]:
# Cell 7: Funksjon for å beregne alle naturpoeng for et datasett
def beregn_alle_naturpoeng(df):
    """
    Beregner alle naturpoeng-verdier for hvert delområde i datasettet
    og legger disse til som kolonner i dataframen.
    """
    # Opprett kolonner for beregnede verdier
    df['Naturpoeng_før'] = 0.0
    df['Naturpoeng_direkte_tap'] = 0.0
    df['Naturpoeng_indirekte_tap'] = 0.0
    df['Naturpoeng_tapt'] = 0.0
    df['Naturpoeng_skapt'] = 0.0
    df['Naturpoeng_etter'] = 0.0
    df['Netto_endring'] = 0.0
    df['Prosent_endring'] = 0.0

    # Gå gjennom hver rad og beregn verdier
    for i, row in df.iterrows():
        # Beregn naturpoeng før inngrep
        df.at[i, 'Naturpoeng_før'] = beregn_naturpoeng_før(
            row['Utstrekning'], row['Naturkvalitet'], row['Forvaltningsinteresse']
        )

        # Beregn naturpoeng tapt (direkte og indirekte)
        rest_utstrekning = row['Utstrekning'] - row['Tapt_direkte']
        naturpoeng_tapt, direkte_tap, indirekte_tap = beregn_naturpoeng_tapt(
            row['Tapt_direkte'], rest_utstrekning, row['Naturkvalitet'],
            row['Forvaltningsinteresse'], row['Påvirkning']
        )
        df.at[i, 'Naturpoeng_tapt'] = naturpoeng_tapt
        df.at[i, 'Naturpoeng_direkte_tap'] = direkte_tap
        df.at[i, 'Naturpoeng_indirekte_tap'] = indirekte_tap

        # Beregn naturpoeng skapt fra kompensasjon
        er_offsite = row['Komp_type'] == 'Off-site'
        original_areal = row['Original_areal'] if er_offsite else None

        if row['Komp_type'] != '-':
            df.at[i, 'Naturpoeng_skapt'] = beregn_naturpoeng_skapt(
                row['Komp_utstrekning'], row['Komp_naturkvalitet'], row['Komp_forvaltningsinteresse'],
                row['Risiko_avstand'], row['Risiko_tid'], row['Risiko_vanskelighet'],
                er_offsite=er_offsite, original_areal=original_areal
            )

        # Beregn naturpoeng etter inngrep og endringer
        df.at[i, 'Naturpoeng_etter'] = df.at[i, 'Naturpoeng_før'] - df.at[i, 'Naturpoeng_tapt'] + df.at[i, 'Naturpoeng_skapt']
        df.at[i, 'Netto_endring'] = df.at[i, 'Naturpoeng_etter'] - df.at[i, 'Naturpoeng_før']

        if df.at[i, 'Naturpoeng_før'] > 0:
            df.at[i, 'Prosent_endring'] = (df.at[i, 'Netto_endring'] / df.at[i, 'Naturpoeng_før']) * 100

    return df

In [7]:
# Cell 8: Hjelpefunksjoner
def get_unit(delregnskap):
    """Returnerer målenhet basert på delregnskap"""
    if delregnskap == 'Vannforekomster':
        return 'meter'
    else:
        return 'dekar'

In [8]:
# Cell 9: Funksjon for å vise oversiktspanel
def vis_oversiktspanel(df):
    """
    Viser oversiktspanelet med hovedtall for hele prosjektet,
    inkludert grafikk og tabeller.
    """
    # Beregn nøkkeltall for hele prosjektet
    total_før = df['Naturpoeng_før'].sum()
    total_tapt = df['Naturpoeng_tapt'].sum()
    total_skapt = df['Naturpoeng_skapt'].sum()
    total_etter = df['Naturpoeng_etter'].sum()
    netto_endring = total_etter - total_før
    prosent_endring = (netto_endring / total_før) * 100 if total_før > 0 else 0

    # Beregn nøkkeltall per delregnskap
    delregnskap_grupper = df.groupby('Delregnskap').agg({
        'Naturpoeng_før': 'sum',
        'Naturpoeng_tapt': 'sum',
        'Naturpoeng_skapt': 'sum',
        'Naturpoeng_etter': 'sum'
    })

    delregnskap_grupper['Netto_endring'] = delregnskap_grupper['Naturpoeng_etter'] - delregnskap_grupper['Naturpoeng_før']
    delregnskap_grupper['Prosent_endring'] = (delregnskap_grupper['Netto_endring'] / delregnskap_grupper['Naturpoeng_før']) * 100

    # Lag figurer
    fig = plt.figure(figsize=(15, 10))
    gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1], height_ratios=[1, 1])

    # 1. Totaloversikt stolpediagram
    ax1 = plt.subplot(gs[0, 0])
    x_pos = [0, 1, 2, 3]
    bars = ax1.bar(x_pos, [total_før, total_tapt, total_skapt, total_etter],
           color=['green', 'red', 'blue', 'purple'])

    # Legg til verdier over søylene
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{int(height)}',
                 ha='center', va='bottom', fontweight='bold')

    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(['Før inngrep', 'Tapt', 'Skapt', 'Etter inngrep'])
    ax1.set_title('Totalt naturregnskap for hele prosjektet', fontsize=14)
    ax1.set_ylabel('Naturpoeng')

    # Legg til netto endring som tekst
    endring_farge = 'green' if netto_endring >= 0 else 'red'
    ax1.text(1.5, total_før * 0.5,
             f'Netto endring: {netto_endring:.1f} poeng ({prosent_endring:.1f}%)',
             fontsize=12, fontweight='bold', color=endring_farge,
             ha='center', bbox=dict(facecolor='white', alpha=0.8))

    # 2. Tårtediagram for fordeling av naturpoeng før
    ax2 = plt.subplot(gs[0, 1])
    delregnskap_labels = delregnskap_grupper.index
    delregnskap_verdier = delregnskap_grupper['Naturpoeng_før']

    ax2.pie(delregnskap_verdier, labels=delregnskap_labels, autopct='%1.1f%%',
            shadow=False, startangle=90, colors=['lightgreen', 'lightblue', 'lightsalmon'])
    ax2.set_title('Fordeling av naturpoeng før inngrep', fontsize=14)

    # 3. Delregnskap sammenligning
    ax3 = plt.subplot(gs[1, :])
    bar_width = 0.2
    index = np.arange(len(delregnskap_grupper))

    b1 = ax3.bar(index, delregnskap_grupper['Naturpoeng_før'], bar_width,
                label='Før inngrep', color='green')
    b2 = ax3.bar(index + bar_width, delregnskap_grupper['Naturpoeng_tapt'], bar_width,
                label='Tapt', color='red')
    b3 = ax3.bar(index + bar_width*2, delregnskap_grupper['Naturpoeng_skapt'], bar_width,
                label='Skapt', color='blue')
    b4 = ax3.bar(index + bar_width*3, delregnskap_grupper['Naturpoeng_etter'], bar_width,
                label='Etter inngrep', color='purple')

    # Legg til verdier på søylene
    for i, bars in enumerate([b1, b2, b3, b4]):
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                ax3.text(bar.get_x() + bar.get_width()/2., height + 5,
                        f'{int(height)}', ha='center', va='bottom', fontsize=9)

    ax3.set_xticks(index + bar_width * 1.5)
    ax3.set_xticklabels(delregnskap_grupper.index)
    ax3.legend(loc='upper right')
    ax3.set_title('Sammenligning av naturpoeng per delregnskap', fontsize=14)
    ax3.set_ylabel('Naturpoeng')

    plt.tight_layout()
    plt.show()

    # Vis nøkkeltall i tekstformat
    display(Markdown("## 📊 Nøkkeltall for hele prosjektet"))
    display(Markdown(f"""
    | Indikator | Verdi | Endring |
    |---|---|---|
    | **Naturpoeng før inngrep** | {total_før:.1f} | - |
    | **Naturpoeng tapt** | {total_tapt:.1f} | (-{(total_tapt/total_før*100):.1f}%) |
    | **Naturpoeng skapt** | {total_skapt:.1f} | (+{(total_skapt/total_før*100):.1f}%) |
    | **Naturpoeng etter inngrep** | {total_etter:.1f} | - |
    | **Netto endring** | {netto_endring:.1f} | ({prosent_endring:.1f}%) |
    """))

    # Vis nøkkeltall per delregnskap
    display(Markdown("## 📊 Nøkkeltall per delregnskap"))

    # Formater delregnskapstabellen
    delregnskap_tabell = delregnskap_grupper.copy()
    delregnskap_tabell['Prosent_endring'] = delregnskap_tabell['Prosent_endring'].map('{:+.1f}%'.format)

    # Vis tabellen
    display(HTML(delregnskap_tabell.to_html(float_format=lambda x: f'{x:.1f}')))

In [9]:
# Cell 10: Funksjon for å vise detaljpanel for enkeltområder
def vis_detaljpanel(df, selected_id=None):
    """
    Viser detaljert informasjon for ett spesifikt delområde
    eller en oversikt over alle delområder.

    Args:
        df: DataFrame med alle prosjektdata
        selected_id: ID til det valgte delområdet, eller None for oversikt
    """
    if selected_id:
        row = df[df['ID'] == selected_id].iloc[0]

        # Beregn detaljer for det valgte området
        rest_utstrekning = row['Utstrekning'] - row['Tapt_direkte']
        er_offsite = row['Komp_type'] == 'Off-site'

        # Lage figur
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # 1. Stolpediagram med detaljer
        x_pos = [0, 1, 2, 3, 4]
        data = [
            row['Naturpoeng_før'],
            row['Naturpoeng_direkte_tap'],
            row['Naturpoeng_indirekte_tap'],
            row['Naturpoeng_skapt'],
            row['Naturpoeng_etter']
        ]

        colors = ['green', 'darkred', 'indianred', 'blue', 'purple']
        labels = ['Før inngrep', 'Direkte tap', 'Indirekte tap', 'Skapt', 'Etter inngrep']

        bars = ax1.bar(x_pos, data, color=colors)

        # Legg til verdier over søylene
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.1f}',
                    ha='center', va='bottom', fontweight='bold')

        ax1.set_xticks(x_pos)
        ax1.set_xticklabels(labels)
        ax1.set_title(f'Detaljert naturregnskap for {row["Navn"]}', fontsize=14)
        ax1.set_ylabel('Naturpoeng')

        # 2. Ringdiagram som viser visuell fordeling av bruk og kompensasjon
        if row['Komp_type'] != '-':
            sizes = [
                row['Tapt_direkte'],  # Direkte tap
                rest_utstrekning,     # Gjenværende område
                row['Komp_utstrekning']  # Kompensasjonsområde
            ]

            labels = [
                f'Direkte tapt ({row["Tapt_direkte"]} {get_unit(row["Delregnskap"])})',
                f'Gjenværende ({rest_utstrekning} {get_unit(row["Delregnskap"])})',
                f'Kompensasjon ({row["Komp_utstrekning"]} {get_unit(row["Delregnskap"])})'
            ]

            colors = ['red', 'lightgreen', 'blue']

            # Lag ringdiagram med hull i midten
            ax2.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
                   shadow=False, startangle=90, wedgeprops={'width': 0.5})

            ax2.set_title(f'Fordeling av areal for {row["Navn"]}', fontsize=14)
        else:
            sizes = [
                row['Tapt_direkte'],  # Direkte tap
                rest_utstrekning      # Gjenværende område
            ]

            labels = [
                f'Direkte tapt ({row["Tapt_direkte"]} {get_unit(row["Delregnskap"])})',
                f'Gjenværende ({rest_utstrekning} {get_unit(row["Delregnskap"])})'
            ]

            colors = ['red', 'lightgreen']

            # Lag ringdiagram med hull i midten
            ax2.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
                   shadow=False, startangle=90, wedgeprops={'width': 0.5})

            ax2.set_title(f'Fordeling av areal for {row["Navn"]}', fontsize=14)

        plt.tight_layout()
        plt.show()

        # Vis nøkkeltall for området
        display(Markdown(f"## 📋 Detaljer for {row['Navn']} ({row['ID']})"))
        display(Markdown(f"""
        | Parameter | Verdi |
        |---|---|
        | **Delregnskap** | {row['Delregnskap']} |
        | **Naturtype** | {row['Naturtype']} |
        | **Utstrekning** | {row['Utstrekning']} {get_unit(row['Delregnskap'])} |
        | **Naturkvalitet** | {row['Naturkvalitet']} |
        | **Forvaltningsinteresse** | {row['Forvaltningsinteresse']} |
        | **Tapt direkte** | {row['Tapt_direkte']} {get_unit(row['Delregnskap'])} |
        | **Påvirkning på gjenværende** | {row['Påvirkning']} |
        | **Kompensasjonstype** | {row['Komp_type']} |
        | **Kompensasjon utstrekning** | {row['Komp_utstrekning']} {get_unit(row['Delregnskap'])} |
        | **Kompensasjon naturkvalitet** | {row['Komp_naturkvalitet']} |
        | **Kompensasjon forvaltningsinteresse** | {row['Komp_forvaltningsinteresse']} |
        | **Risiko avstand** | {row['Risiko_avstand']} |
        | **Risiko tid** | {row['Risiko_tid']} |
        | **Risiko vanskelighet** | {row['Risiko_vanskelighet']} |
        """))

        display(Markdown(f"## 📊 Naturpoeng for {row['Navn']} ({row['ID']})"))
        display(Markdown(f"""
        | Indikator | Verdi | Prosentandel |
        |---|---|---|
        | **Naturpoeng før inngrep** | {row['Naturpoeng_før']:.1f} | 100% |
        | **Naturpoeng direkte tap** | {row['Naturpoeng_direkte_tap']:.1f} | {row['Naturpoeng_direkte_tap']/row['Naturpoeng_før']*100:.1f}% |
        | **Naturpoeng indirekte tap** | {row['Naturpoeng_indirekte_tap']:.1f} | {row['Naturpoeng_indirekte_tap']/row['Naturpoeng_før']*100:.1f}% |
        | **Naturpoeng totalt tapt** | {row['Naturpoeng_tapt']:.1f} | {row['Naturpoeng_tapt']/row['Naturpoeng_før']*100:.1f}% |
        | **Naturpoeng skapt** | {row['Naturpoeng_skapt']:.1f} | {row['Naturpoeng_skapt']/row['Naturpoeng_før']*100:.1f}% |
        | **Naturpoeng etter inngrep** | {row['Naturpoeng_etter']:.1f} | - |
        | **Netto endring** | {row['Netto_endring']:.1f} | {row['Prosent_endring']:.1f}% |
        """))
    else:
        # Sortert opp og ned etter netto endring
        df_sorted = df.sort_values('Prosent_endring')

        # Vis top 10 verste og beste delområder
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

        # 1. Topp 10 verste områder
        worst_10 = df_sorted.head(10)
        ax1.barh(worst_10['Navn'], worst_10['Prosent_endring'], color='red')
        ax1.set_title('Topp 10 områder med størst negativ endring', fontsize=14)
        ax1.set_xlabel('Endring i naturpoeng (%)')
        # Legg til verdier på søylene
        for i, v in enumerate(worst_10['Prosent_endring']):
            ax1.text(v - 1, i, f'{v:.1f}%', ha='right', va='center',
                    color='white', fontweight='bold')

        # 2. Topp 10 beste områder
        best_10 = df_sorted.tail(10).iloc[::-1]  # Reverse order
        ax2.barh(best_10['Navn'], best_10['Prosent_endring'], color='green')
        ax2.set_title('Topp 10 områder med best eller minst negativ endring', fontsize=14)
        ax2.set_xlabel('Endring i naturpoeng (%)')
        # Legg til verdier på søylene
        for i, v in enumerate(best_10['Prosent_endring']):
            color = 'black' if v < 0 else 'white'
            ax2.text(v + 0.5, i, f'{v:.1f}%', ha='left', va='center',
                    color=color, fontweight='bold')

        plt.tight_layout()
        plt.show()

        # Vis tabell med alle delområder
        display(Markdown("## 📋 Oversikt over alle delområder"))
        # Filtrer og formater dataframe for visning
        df_display = df[['ID', 'Navn', 'Delregnskap', 'Naturpoeng_før',
                         'Naturpoeng_tapt', 'Naturpoeng_skapt', 'Naturpoeng_etter',
                         'Netto_endring', 'Prosent_endring']]

        df_display = df_display.sort_values(['Delregnskap', 'Prosent_endring'])
        # Format prosent-kolonnen
        df_display['Prosent_endring'] = df_display['Prosent_endring'].map('{:+.1f}%'.format)

        # Vis tabellen
        display(HTML(df_display.to_html(float_format=lambda x: f'{x:.1f}')))

In [10]:
# Cell 11: Funksjon for å vise kartpanel
def vis_kartpanel(df):
    """
    Viser en stilisert kartvisning av delområdene.
    I en faktisk implementasjon ville dette vært koblet til geografiske data.
    """
    # I en ekte implementasjon ville dette panelet koblet til geografiske data
    # og vist et interaktivt kart med naturtyper, inngrep og kompensasjonstiltak.

    # For nå lager vi en stilisert visualisering som et eksempel
    fig, ax = plt.subplots(figsize=(12, 10))

    # Fordel punkter etter delregnskap (dummy posisjon)
    np.random.seed(42)  # For reproduserbarhet
    x_oko = np.random.rand(len(df[df['Delregnskap'] == 'Økosystemareal'])) * 10
    y_oko = np.random.rand(len(df[df['Delregnskap'] == 'Økosystemareal'])) * 10

    x_funk = np.random.rand(len(df[df['Delregnskap'] == 'Økologiske funksjonsområder'])) * 10 + 3
    y_funk = np.random.rand(len(df[df['Delregnskap'] == 'Økologiske funksjonsområder'])) * 10 - 3

    x_vann = np.random.rand(len(df[df['Delregnskap'] == 'Vannforekomster'])) * 10 - 2
    y_vann = np.random.rand(len(df[df['Delregnskap'] == 'Vannforekomster'])) * 10 + 2

    # Størrelse basert på naturpoeng før
    s_oko = df[df['Delregnskap'] == 'Økosystemareal']['Naturpoeng_før'] * 5
    s_funk = df[df['Delregnskap'] == 'Økologiske funksjonsområder']['Naturpoeng_før'] * 5
    s_vann = df[df['Delregnskap'] == 'Vannforekomster']['Naturpoeng_før'] * 5

    # Farge basert på prosentvis endring
    c_oko = df[df['Delregnskap'] == 'Økosystemareal']['Prosent_endring']
    c_funk = df[df['Delregnskap'] == 'Økologiske funksjonsområder']['Prosent_endring']
    c_vann = df[df['Delregnskap'] == 'Vannforekomster']['Prosent_endring']

    # Plott punktene
    scatter1 = ax.scatter(x_oko, y_oko, s=s_oko, c=c_oko, cmap=change_cmap,
                         vmin=-20, vmax=5, alpha=0.8, edgecolors='black')
    scatter2 = ax.scatter(x_funk, y_funk, s=s_funk, c=c_funk, cmap=change_cmap,
                         vmin=-20, vmax=5, alpha=0.8, edgecolors='black', marker='s')
    scatter3 = ax.scatter(x_vann, y_vann, s=s_vann, c=c_vann, cmap=change_cmap,
                         vmin=-20, vmax=5, alpha=0.8, edgecolors='black', marker='^')

    # Legg til ID-etiketter
    ids_oko = df[df['Delregnskap'] == 'Økosystemareal']['ID'].values
    ids_funk = df[df['Delregnskap'] == 'Økologiske funksjonsområder']['ID'].values
    ids_vann = df[df['Delregnskap'] == 'Vannforekomster']['ID'].values

    for i, txt in enumerate(ids_oko):
        ax.annotate(txt, (x_oko[i], y_oko[i]), xytext=(3, 3), textcoords='offset points')

    for i, txt in enumerate(ids_funk):
        ax.annotate(txt, (x_funk[i], y_funk[i]), xytext=(3, 3), textcoords='offset points')

    for i, txt in enumerate(ids_vann):
        ax.annotate(txt, (x_vann[i], y_vann[i]), xytext=(3, 3), textcoords='offset points')

    # Legg til colorbar for prosent endring
    cbar = plt.colorbar(scatter1)
    cbar.set_label('Endring i naturpoeng (%)')

    # Legg til forklaringer
    ax.set_title('Visualisering av delområder (stilisert kart)', fontsize=16)

    # Legg til legend for symboler
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='gray',
                  markersize=15, label='Økosystemareal'),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='gray',
                  markersize=15, label='Økologiske funksjonsområder'),
        plt.Line2D([0], [0], marker='^', color='w', markerfacecolor='gray',
                  markersize=15, label='Vannforekomster')
    ]

    ax.legend(handles=legend_elements, loc='upper right')

    # Fjern aksene for et rent kart-look
    ax.set_xticks([])
    ax.set_yticks([])

    # Legg til en undertekst som forklarer at dette er et stilisert kart
    plt.figtext(0.5, 0.01, "Dette er en stilisert visualisering. I et faktisk dashboard ville dette vært et interaktivt kart med faktiske geografiske data.",
               ha='center', fontsize=10, bbox=dict(facecolor='lightgray', alpha=0.5))

    plt.tight_layout()
    plt.show()

    # Vis litt forklarende tekst
    display(Markdown("""
    ## 🗺️ Kartforklaring

    I dette dummykartet representerer:
    - **Sirklene** økosystemarealene
    - **Kvadratene** økologiske funksjonsområder
    - **Trekantene** vannforekomster

    **Størrelsen** på symbolene representerer naturpoeng før inngrep, mens **fargen**
    representerer prosentvis endring (fra rødt = negativt til blått = positivt).

    I et faktisk dashboard ville dette vært et interaktivt GIS-kart med faktiske geografiske
    data, hvor du kunne zoome inn/ut og klikke på områder for å se detaljer.
    """))

In [11]:
# Cell 12: Funksjon for å vise tiltakspanel
def vis_tiltakspanel(df):
    """
    Viser oversikt over kompensasjonstiltak, prioriterte områder
    og anbefalinger for videre tiltak.
    """
    # Sorter delområder etter prosent endring for å identifisere prioritetsområder
    df_sorted = df.sort_values('Prosent_endring')

    # Gruppere delområder etter kompensasjonstype
    komp_fordeling = df['Komp_type'].value_counts()

    # Beregn statistikk om kompensasjonstiltak
    kompensasjon_stats = {
        'total_områder': len(df),
        'områder_med_komp': len(df[df['Komp_type'] != '-']),
        'områder_uten_komp': len(df[df['Komp_type'] == '-']),
        'områder_onsite': len(df[df['Komp_type'] == 'On-site']),
        'områder_offsite': len(df[df['Komp_type'] == 'Off-site']),
        'gj_risiko_avstand': df[df['Komp_type'] != '-']['Risiko_avstand'].mean(),
        'gj_risiko_tid': df[df['Komp_type'] != '-']['Risiko_tid'].mean(),
        'gj_risiko_vanskelighet': df[df['Komp_type'] != '-']['Risiko_vanskelighet'].mean(),
        'total_skaping': df['Naturpoeng_skapt'].sum(),
        'total_tap': df['Naturpoeng_tapt'].sum(),
        'kompensasjonsgrad': df['Naturpoeng_skapt'].sum() / df['Naturpoeng_tapt'].sum() * 100 if df['Naturpoeng_tapt'].sum() > 0 else 0
    }

    # 1. Figur som viser fordelingen av kompensasjonstyper
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    komp_fordeling.plot.pie(ax=ax1, autopct='%1.1f%%', startangle=90,
                          colors=['lightgreen', 'lightblue', 'lightgray'],
                          labels=['On-site', 'Off-site', 'Ingen kompensasjon'],
                          wedgeprops={'width': 0.5})
    ax1.set_title('Fordeling av kompensasjonstyper', fontsize=14)
    ax1.set_ylabel('')

    # 2. Figur som viser kompensasjonsgraden for hvert delregnskap
    delregnskap_komp = df.groupby('Delregnskap').agg({
        'Naturpoeng_tapt': 'sum',
        'Naturpoeng_skapt': 'sum'
    })

    delregnskap_komp['Kompensasjonsgrad'] = delregnskap_komp['Naturpoeng_skapt'] / delregnskap_komp['Naturpoeng_tapt'] * 100

    bar_colors = ['lightgreen' if x >= 100 else 'orange' if x >= 80 else 'red' for x in delregnskap_komp['Kompensasjonsgrad']]

    delregnskap_komp['Kompensasjonsgrad'].plot.bar(ax=ax2, color=bar_colors)
    ax2.set_title('Kompensasjonsgrad per delregnskap', fontsize=14)
    ax2.set_ylabel('Kompensasjonsgrad (%)')
    ax2.axhline(y=100, color='green', linestyle='--')

    # Legg til verdier på søylene
    for i, v in enumerate(delregnskap_komp['Kompensasjonsgrad']):
        ax2.text(i, v + 3, f'{v:.1f}%', ha='center')

    plt.tight_layout()
    plt.show()

    # Vis nøkkeltall om tiltakene
    display(Markdown("## 📋 Nøkkeltall for kompensasjonstiltak"))
    display(Markdown(f"""
    | Indikator | Verdi |
    |---|---|
    | **Antall delområder totalt** | {kompensasjon_stats['total_områder']} |
    | **Områder med kompensasjon** | {kompensasjon_stats['områder_med_komp']} ({kompensasjon_stats['områder_med_komp']/kompensasjon_stats['total_områder']*100:.1f}%) |
    | **Områder uten kompensasjon** | {kompensasjon_stats['områder_uten_komp']} ({kompensasjon_stats['områder_uten_komp']/kompensasjon_stats['total_områder']*100:.1f}%) |
    | **On-site kompensasjon** | {kompensasjon_stats['områder_onsite']} områder |
    | **Off-site kompensasjon** | {kompensasjon_stats['områder_offsite']} områder |
    | **Gjennomsnittlig risiko avstand** | {kompensasjon_stats['gj_risiko_avstand']:.2f} |
    | **Gjennomsnittlig risiko tid** | {kompensasjon_stats['gj_risiko_tid']:.2f} |
    | **Gjennomsnittlig risiko vanskelighet** | {kompensasjon_stats['gj_risiko_vanskelighet']:.2f} |
    | **Total naturpoeng tapt** | {kompensasjon_stats['total_tap']:.1f} |
    | **Total naturpoeng skapt** | {kompensasjon_stats['total_skaping']:.1f} |
    | **Samlet kompensasjonsgrad** | {kompensasjon_stats['kompensasjonsgrad']:.1f}% |
    """))

    # Identifiser prioriterte områder for ytterligere tiltak
    worst_areas = df_sorted.head(5)

    display(Markdown("## 🚨 Prioriterte områder for ytterligere tiltak"))
    for i, (idx, row) in enumerate(worst_areas.iterrows()):
        display(Markdown(f"""
        ### {i+1}. {row['Navn']} ({row['ID']})
        - **Delregnskap**: {row['Delregnskap']}
        - **Naturtype**: {row['Naturtype']}
        - **Netto endring**: {row['Netto_endring']:.1f} naturpoeng ({row['Prosent_endring']:.1f}%)
        - **Nåværende kompensasjonstype**: {row['Komp_type']}
        - **Mulige forbedringer**:
          - {'Øke kompensasjonsareal' if row['Komp_type'] != '-' else 'Implementere kompensasjonstiltak'}
          - {'Forbedre naturkvalitet i kompensasjonsområdet' if row['Komp_type'] != '-' else 'Redusere påvirkning på gjenværende område'}
          - {'Redusere risikofaktorer' if row['Komp_type'] != '-' else 'Vurdere on-site restaurering'}
        """))

In [12]:
# Cell 13: Funksjon for å vise rapporteringspanel
def vis_rapporteringspanel(df):
    """
    Viser et sammendrag av naturregnskapet egnet for rapportering
    og kommunikasjon til prosjektledelse og interessenter.
    """
    # Beregn nøkkeltall for hele prosjektet
    total_før = df['Naturpoeng_før'].sum()
    total_tapt = df['Naturpoeng_tapt'].sum()
    total_skapt = df['Naturpoeng_skapt'].sum()
    total_etter = df['Naturpoeng_etter'].sum()
    netto_endring = total_etter - total_før
    prosent_endring = (netto_endring / total_før) * 100 if total_før > 0 else 0

    # Beregn kompensasjonsgrad
    kompensasjonsgrad = total_skapt / total_tapt * 100 if total_tapt > 0 else 0

    # Lag oppsummeringsgrafikk
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # 1. Vannfallsdiagram som viser endringer
    x_labels = ['Før\ninngrep', 'Direkte\ntap', 'Indirekte\ntap', 'Skapt', 'Etter\ninngrep']

    # Beregn totalt direkte og indirekte tap
    total_direkte = df['Naturpoeng_direkte_tap'].sum()
    total_indirekte = df['Naturpoeng_indirekte_tap'].sum()

    # Waterfall chart
    ax1.bar(0, total_før, color='green', width=0.5)
    ax1.bar(1, -total_direkte, color='darkred', width=0.5)
    ax1.bar(2, -total_indirekte, color='indianred', width=0.5)
    ax1.bar(3, total_skapt, color='blue', width=0.5)
    ax1.bar(4, total_etter, color='purple', width=0.5)

    # Legg til verdier
    ax1.text(0, total_før/2, f'{total_før:.0f}', ha='center', va='center', color='white', fontweight='bold')
    ax1.text(1, -total_direkte/2, f'-{total_direkte:.0f}', ha='center', va='center', color='white', fontweight='bold')
    ax1.text(2, -total_indirekte/2, f'-{total_indirekte:.0f}', ha='center', va='center', color='white', fontweight='bold')
    ax1.text(3, total_skapt/2, f'+{total_skapt:.0f}', ha='center', va='center', color='white', fontweight='bold')
    ax1.text(4, total_etter/2, f'{total_etter:.0f}', ha='center', va='center', color='white', fontweight='bold')

    ax1.set_xticks(range(5))
    ax1.set_xticklabels(x_labels)
    ax1.set_title('Samlet naturregnskap for prosjektet', fontsize=14)
    ax1.set_ylabel('Naturpoeng')

    # Legg til netto endring som tekst
    endring_farge = 'green' if netto_endring >= 0 else 'red'
    ax1.text(2, total_før * 0.7,
             f'Netto endring: {netto_endring:.1f} poeng ({prosent_endring:.1f}%)',
             fontsize=12, fontweight='bold', color=endring_farge,
             ha='center', bbox=dict(facecolor='white', alpha=0.8))

    # 2. Kompensasjonsgrad i forhold til målsetninger
    # Doughnut chart
    kompensasjonsgrader = [min(kompensasjonsgrad, 100), max(0, 100 - min(kompensasjonsgrad, 100))]
    labels = [f'Oppnådd: {kompensasjonsgrad:.1f}%', f'Gjenstående: {max(0, 100 - kompensasjonsgrad):.1f}%']
    wedgeprops = {'width':0.3, 'edgecolor':'w', 'linewidth':3}

    if kompensasjonsgrad >= 100:
        colors = ['green', 'lightgray']
        ax2.pie(kompensasjonsgrader, labels=labels, colors=colors, autopct='%1.1f%%',
               startangle=90, wedgeprops=wedgeprops)
    else:
        colors = ['orange', 'lightgray']
        ax2.pie(kompensasjonsgrader, labels=labels, colors=colors, autopct='%1.1f%%',
               startangle=90, wedgeprops=wedgeprops)

    # Legg til tekst om status
    if kompensasjonsgrad >= 100:
        status_text = "✅ Målet om 100% kompensasjon er nådd"
        status_color = 'green'
    elif kompensasjonsgrad >= 80:
        status_text = "⚠️ Nær målet, men ikke helt i mål"
        status_color = 'orange'
    else:
        status_text = "❌ Målet om 100% kompensasjon er ikke nådd"
        status_color = 'red'

    ax2.text(0, -1.5, status_text, fontsize=12, fontweight='bold', color=status_color, ha='center')
    ax2.set_title('Kompensasjonsgrad i forhold til mål (100%)', fontsize=14)

    plt.tight_layout()
    plt.show()

    # Generer rapport i tekstformat
    display(Markdown("# 📑 Oppsummeringsrapport for naturregnskap"))

    # Prosjektoppsummering
    display(Markdown("## 📊 Prosjektoppsummering"))
    display(Markdown(f"""
    Dette naturregnskapet oppsummerer konsekvensene av prosjektet på tvers av 50 delområder,
    fordelt på tre delregnskap: Økosystemareal ({len(df[df['Delregnskap'] == 'Økosystemareal'])} områder),
    Økologiske funksjonsområder ({len(df[df['Delregnskap'] == 'Økologiske funksjonsområder'])} områder) og
    Vannforekomster ({len(df[df['Delregnskap'] == 'Vannforekomster'])} områder).

    ### Nøkkeltall:
    - **Naturpoeng før inngrep**: {total_før:.1f}
    - **Naturpoeng tapt**: {total_tapt:.1f} (direkte tap: {total_direkte:.1f}, indirekte tap: {total_indirekte:.1f})
    - **Naturpoeng skapt gjennom tiltak**: {total_skapt:.1f}
    - **Naturpoeng etter inngrep**: {total_etter:.1f}
    - **Netto endring**: {netto_endring:.1f} poeng ({prosent_endring:.1f}%)
    - **Kompensasjonsgrad**: {kompensasjonsgrad:.1f}% av tapte naturpoeng

    ### Kompensasjonstiltak:
    - **Områder med kompensasjon**: {len(df[df['Komp_type'] != '-'])} av {len(df)} ({len(df[df['Komp_type'] != '-'])/len(df)*100:.1f}%)
    - **On-site kompensasjon**: {len(df[df['Komp_type'] == 'On-site'])} områder
    - **Off-site kompensasjon**: {len(df[df['Komp_type'] == 'Off-site'])} områder
    - **Gjennomsnittlig risikofaktor (avstand)**: {df[df['Komp_type'] != '-']['Risiko_avstand'].mean():.2f}
    - **Gjennomsnittlig risikofaktor (tid)**: {df[df['Komp_type'] != '-']['Risiko_tid'].mean():.2f}
    - **Gjennomsnittlig risikofaktor (vanskelighet)**: {df[df['Komp_type'] != '-']['Risiko_vanskelighet'].mean():.2f}
    """))

    # Konklusjon og anbefalinger
    display(Markdown("## 🔍 Konklusjon og anbefalinger"))

    if prosent_endring >= 0:
        konklusjon = "Prosjektet oppnår en netto positiv eller nøytral effekt på naturverdier. Dette er en svært god prestasjon som bør fremheves."
    elif prosent_endring >= -10:
        konklusjon = "Prosjektet har en moderat negativ innvirkning på naturverdiene, men kompensasjonstiltakene reduserer effekten betydelig."
    else:
        konklusjon = "Prosjektet har en vesentlig negativ innvirkning på naturverdiene, til tross for kompensasjonstiltak."

    if kompensasjonsgrad >= 100:
        komp_vurdering = "Kompensasjonsgraden på over 100% viser at tiltakene mer enn oppveier de negative effektene."
    elif kompensasjonsgrad >= 80:
        komp_vurdering = f"Kompensasjonsgraden på {kompensasjonsgrad:.1f}% er nær målet på 100%, men ytterligere tiltak bør vurderes."
    else:
        komp_vurdering = f"Kompensasjonsgraden på kun {kompensasjonsgrad:.1f}% er vesentlig under målet på 100%. Det anbefales sterkt å implementere ytterligere tiltak."

    # Finn de verste områdene
    worst_areas = df.sort_values('Prosent_endring').head(3)
    problem_areas = ", ".join([f"{row['Navn']} ({row['ID']})" for idx, row in worst_areas.iterrows()])

    display(Markdown(f"""
    ### Konklusjon:
    {konklusjon} {komp_vurdering}

    ### Anbefalinger:
    1. **Fokuser på problemområder**: De største negative effektene er i områdene {problem_areas}. Disse bør prioriteres for ytterligere tiltak.

    2. **Forbedre kompensasjonstiltak**: {
    "Vurder å øke kompensasjonsarealet eller forbedre kvaliteten på eksisterende tiltak." if kompensasjonsgrad < 100 else
    "Eksisterende kompensasjonstiltak er tilstrekkelige, men fortsett å overvåke effektiviteten over tid."
    }

    3. **Risikofaktorer**: {
    "Arbeid med å redusere risikofaktorene knyttet til avstand, tid og vanskelighet for å øke kvaliteten på kompensasjonen." if df[df['Komp_type'] != '-']['Risiko_avstand'].mean() < 0.8 or df[df['Komp_type'] != '-']['Risiko_tid'].mean() < 0.8 or df[df['Komp_type'] != '-']['Risiko_vanskelighet'].mean() < 0.8 else
    "Risikofaktorene for kompensasjonstiltakene er relativt lave, noe som er positivt for sannsynligheten for vellykket kompensasjon."
    }

    4. **Overvåking**: Etabler et robust overvåkingsprogram for å følge utviklingen i både påvirkede områder og kompensasjonsområder over tid.

    5. **Dokumentasjon**: Dokumenter erfaringer og resultater for å forbedre framtidige prosjekter og bidra til kunnskapsutvikling innen naturkompensasjon.
    """))

In [13]:
# Cell 14: Dashboard-oppsett
def setup_dashboard():
    """
    Setter opp og viser hele dashboardet med faner
    for navigering mellom de ulike panelene.
    """
    # Last inn og klargjør data
    df = importer_eksempeldata()
    df = beregn_alle_naturpoeng(df)

    # Lag faner for dashbordet
    tab_output = widgets.Output()
    tabs = widgets.Tab()

    tab_titles = [
        '📊 Oversikt',
        '🔍 Detaljer',
        '🗺️ Kart',
        '🛠️ Tiltak',
        '📑 Rapportering'
    ]

    oversikt_output = widgets.Output()
    with oversikt_output:
        vis_oversiktspanel(df)

    # Detaljpanel med områdevalg
    detalj_output = widgets.Output()
    område_dropdown = widgets.Dropdown(
        options=[('Alle områder', None)] + [(row['Navn'] + ' (' + row['ID'] + ')', row['ID']) for i, row in df.iterrows()],
        value=None,
        description='Velg område:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    def on_område_change(change):
        with detalj_output:
            detalj_output.clear_output(wait=True)
            vis_detaljpanel(df, change['new'])

    område_dropdown.observe(on_område_change, names='value')

    detalj_widget = widgets.VBox([
        widgets.HTML("<h2>🔍 Detaljoversikt for delområder</h2>"),
        widgets.HTML("<p>Velg et spesifikt område for detaljert analyse, eller velg 'Alle områder' for å se en oversikt.</p>"),
        område_dropdown,
        detalj_output
    ])

    with detalj_output:
        vis_detaljpanel(df)

    # Kart panel
    kart_output = widgets.Output()
    with kart_output:
        vis_kartpanel(df)

    # Tiltakspanel
    tiltak_output = widgets.Output()
    with tiltak_output:
        vis_tiltakspanel(df)

    # Rapporteringspanel
    rapport_output = widgets.Output()
    with rapport_output:
        vis_rapporteringspanel(df)

    # Opprett faner
    children = [oversikt_output, detalj_widget, kart_output, tiltak_output, rapport_output]
    tabs = widgets.Tab()
    tabs.children = children
    for i, title in enumerate(tab_titles):
        tabs.set_title(i, title)

    # Vis dashbordet
    display(widgets.HTML("<h1>📊 Naturregnskap Dashboard</h1>"))
    display(widgets.HTML("<p>Dette dashbordet gir oversikt og innsikt i naturregnskapet for prosjektet. Bruk fanene nedenfor for å utforske ulike aspekter.</p>"))
    display(tabs)

In [14]:
# Cell 15: Start dashboardet
def start_dashboard():
    """
    Starter naturregnskapsdashboard med alle funksjoner
    """
    setup_dashboard()

# Start dashbordet
start_dashboard()

HTML(value='<h1>📊 Naturregnskap Dashboard</h1>')

HTML(value='<p>Dette dashbordet gir oversikt og innsikt i naturregnskapet for prosjektet. Bruk fanene nedenfor…

In [ ]:
!pip install pandas numpy matplotlib seaborn ipywidgets

/bin/bash: pip: command not found


In [ ]:
!pip install pandas

/bin/bash: pip: command not found
